# PRO3515 - PFSP LS


## Dupla 9

### Nomes: 
### Lucas Hideki Takeuchi Okamura NUSP:  9274315
### Thales Arantes Kerche Nunes   NUSP: 10769372


Importando as bibliotecas necessárias

In [1]:
import xlwings as xw
import gurobipy as grb
import numpy as np

Definindo a classe Job que usaremos no algoritmo

In [2]:
class Job:
    def __init__(self,i,pi,di):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.p_total=sum(self.p)
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness
        self.bestPos=-1 # Esse novo atribuito tem o objetivo de guardar a melhor posição do job, para a ordenação posterior
    
    def __str__(self):
        return 'Job #{}, p = {}'.format(self.i, self.p)

### Reutilizamos as funções disponibilizada pelo professor, com alguns ajustes para ler o número de máquinas e gravar o Runtime e Gap

In [3]:
def leInst(sheet_num):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('B11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
    jobs=[]
    for i in range(n):
        pi=tab1[i]
        jobs.append(Job(i,pi,0))
    
    return m,n,jobs

In [24]:
def gravaSched(i,jobs,Cmax, m, Runtime, Gap):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',m]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['Cmax',Cmax]
    plan.range('H5').value=['Runtime',Runtime]
    plan.range('H6').value=['Gap',Gap]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5']
    plan.range('A11').value=tab

    print()
    print('Cmax (I('+str(i+1)+')): ', Cmax)
    print()
    print()

Função que atualiza os valores dos completion times em cada instância da Classe Job

In [5]:
def calcCmax(m,jobs):     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]
    
    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]
        
    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]
            
    j=0
    for job in jobs:
        job.C=c[j]
        j+=1
    
    return c[-1][-1]

Função responsável por criar o modelo, inserir os constraints e otimizar, retornando a lista ordenada de jobs, o Cmax, Runtime e o Optimal Gap

In [23]:
def schedGurobi(jobs: list, m: int = 5) -> (list, int, float, float):
    
    n = len(jobs)
    
    # Cria o modelo para a intância que será rodada
    mPFSP = grb.Model(name="Permutation Flow Shop Problem")
    x = mPFSP.addVars(n, n, vtype=grb.GRB.BINARY, name='x')
    s = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb=0, name='s')
    c = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb=0, name='c')
    mPFSP.update()

    # coloca as constraints necessárias para a otimização
    mPFSP.addConstrs((s[j,k] - c[j,k] + grb.quicksum(jobs[i].p[k]*x[i,j] for i in range(n)) <= 0 for k in range(m) for j in range(n)), name='r1')
    mPFSP.addConstrs((s[j,k]-c[j-1,k]>=0 for k in range(m) for j in range(1, n)), name='r2')
    mPFSP.addConstrs((s[j,k]-c[j,k-1]>=0 for k in range(1, m) for j in range(n)), name='r3')
    mPFSP.addConstrs((grb.quicksum(x[i,j] for i in range(n)) == 1  for j in range(n)), name='r4')
    mPFSP.addConstrs((grb.quicksum(x[i,j] for j in range(n)) == 1  for i in range(n)), name='r5')
    mPFSP.update()
    
    # define o objetivo
    mPFSP.ModelSense = grb.GRB.MINIMIZE
    mPFSP.setObjective(c[n-1,m-1])
    mPFSP.update()
    
    # otimiza o modelo e recupera as informações relevantes
    mPFSP.optimize()
    runtime = mPFSP.Runtime
    CmaxFinal = round(mPFSP.objVal)
    gap = mPFSP.MIPGap
    
    # pega a posição ideal de cada job do modelo, e salva essa posição no atributo bestPos de cada job
    pos = np.reshape(mPFSP.getVars()[:n*n], (n, n))
    for i in range(n):
        for j in range(n):
            if pos[i, j].x > 0.999:
                jobs[i].bestPos = j
                break
    
    # ordena os jobs de acordo com a ordem final retirada do modelo nas linhas acima
    jobs.sort(key=lambda job: job.bestPos)
    # imprime a ordem final de jobs
    print([job.i+1 for job in jobs])
    
    #verifica se a solução final está de acordo com a calculada pelo nosso código na ordem final, e atualiza os completion times dos jobs da lista
    if calcCmax(m, jobs) == CmaxFinal:
        print("OK!")
    
    return jobs, CmaxFinal, runtime, gap

## Principal

Caso base pra comprovar o funcionamento do algoritmo implementado

In [19]:
tab1 = [[16, 18, 12],[14,10,11],[13,20,15],[19,15,19],[15,16,16]]
jobs=[]
for i in range(len(tab1)):
    pi=tab1[i]
    jobs.append(Job(i,pi,0))

jobs, Cmax, runtime, gap = schedGurobi(jobs, 3)

print([job.i+1 for job in jobs])
print([job.C for job in jobs])
print('Cmax = ', Cmax)
print('Runtime = ', runtime)
print('Gap = ', gap)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 47 rows, 55 columns and 199 nonzeros
Model fingerprint: 0xf7509807
Variable types: 30 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 14 rows and 15 columns
Presolve time: 0.00s
Presolved: 33 rows, 40 columns, 153 nonzeros
Variable types: 10 continuous, 30 integer (25 binary)
Found heuristic solution: objective 115.0000000
Found heuristic solution: objective 112.0000000
Found heuristic solution: objective 109.0000000

Root relaxation: objective 1.043698e+02, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  104.36981    0   18  109.00000  104.36981  4.25%     -    0s
H    0     0                     107.0000

Abre os dois arquivos que vão ser usados

In [25]:
wb1 = xw.Book("xl05 2 A PFSP Cmax CDS.xlsx")
wb2 = xw.Book()

Roda todos os testes:

In [26]:
for sheet in range(10):
    m,n,jobs=leInst(sheet)

    jobs, Cmax, runtime, gap = schedGurobi(jobs)

    gravaSched(sheet, jobs, Cmax, m, runtime, gap)


Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 315 rows, 600 columns and 3350 nonzeros
Model fingerprint: 0xe43d7fc2
Variable types: 200 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 48 rows and 49 columns
Presolve time: 0.09s
Presolved: 267 rows, 551 columns, 3135 nonzeros
Variable types: 129 continuous, 422 integer (400 binary)
Found heuristic solution: objective 1379.0000000
Found heuristic solution: objective 1369.0000000

Root relaxation: objective 1.171000e+03, 883 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1171.00000    0   60 1369.00000 1171.00000  14.5%     -    0s
H    0     0                    1280.0000000 1171.00000  8.52%     -    0

Fecha o workbook de Dados

In [27]:
wb1.close()